In [15]:
import transformers

In [16]:
TRAIN_FILE = "/tmp/semeval24_task3/SemEval-2024_Task3/official_data/Training_data/text/training.json"
VAL_FILE = "/tmp/semeval24_task3/SemEval-2024_Task3/official_data/Training_data/text/testing.json"
TEST_FILE = "/tmp/semeval24_task3/SemEval-2024_Task3/official_data/Evaluation_Data/Subtask_2_test.json"

In [17]:
CACHE_DIR = "/tmp/akshett.jindal/.huggingface_cache/"

In [18]:
import json

with open(TRAIN_FILE) as f:
    train_data = json.load(f)
with open(VAL_FILE) as f:
    val_data = json.load(f)
with open(TEST_FILE) as f:
    test_data = json.load(f)

In [19]:
len(train_data), len(test_data), len(val_data)

(1236, 665, 138)

In [20]:
# ROBERTA_CONFIG = "/scratch/semeval24_task3/finetuned_models/deberta-v3-base-seq-classifier-finetuned-emotion/checkpoint-27585"
# ROBERTA_CONFIG = "SamLowe/roberta-base-go_emotions"
ROBERTA_CONFIG = "FacebookAI/roberta-large"

In [21]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(ROBERTA_CONFIG, cache_dir=CACHE_DIR)
model = AutoModel.from_pretrained(ROBERTA_CONFIG, cache_dir=CACHE_DIR)

In [22]:
import torch

device = torch.device("cuda:3")

In [23]:
_ = model.to(device)

In [24]:
final_embeddings = {}

In [25]:
from tqdm.auto import tqdm

for data in [train_data, val_data, test_data]:
    for conversation in tqdm(data):
        for utterance in conversation["conversation"]:
            text = utterance["text"]
            video_id = utterance["video_name"]
            # if video_id in final_embeddings:
            #     print("dhoka hua!!!!!!!!!!!!!!!")

            inputs = tokenizer(
                text, return_tensors="pt",
                padding=False, truncation=True,
                max_length=512
            )
            inputs.to(device)

            with torch.no_grad():
                outputs = model(**inputs)

            embeddings = torch.mean(outputs.last_hidden_state, dim=1)

            final_embeddings[video_id] = embeddings.cpu()

100%|██████████| 665/665 [01:53<00:00,  5.86it/s]


In [26]:
len(final_embeddings), list(final_embeddings.items())[0][1].shape

(19920, torch.Size([1, 768]))

In [27]:
for k, v in final_embeddings.items():
    final_embeddings[k] = v.squeeze().cpu().numpy()
    # print(v.squeeze().cpu().numpy())
    # break

In [28]:
import pickle

with open("/tmp/semeval24_task3/text_embeddings/text_embeddings_roberta_large.pkl", "wb") as f:
    pickle.dump(final_embeddings, f)